## 4. Predicción de fugas de clientes en una compañía telefónica (7 puntos)

Una compañía telefónica está interesada en que desarrollemos un modelo que prediga los **100 clientes actuales** (dataset de explotaación) que tienen más probabilidad de abandonar la compañía. Para ello nos proporcionan una base de datos **fuga_clientes_empresa_telefonica_construccion.csv** con casos etiquetados, que usaremos para construir nuestro modelo de predicción.

Los campos de esta base de datos son:

* **Customer ID**

* **network_age**: antigüedad del cliente en días

* **Customer tenure in months:** antigüedad del cliente en meses

* **Total Spend in Months 1 and 2:** gasto total del cliente en los meses de referencia 1 y 2

* **Total SMS Spend:** gasto total en SMS

* **Total Data Spend:** gasto total en datos/internet

* **Total Data Consumption:** consumo total de datos (en KB) durante el período de estudio

* **Total Unique Calls:** número total de llamadas únicas

* **Total Onnet spend:** gasto total en llamadas a otros usuarios de la misma red de telefonía

* **Total Offnet spend:** gasto total en llamadas a otros usuarios de redes de telefonía diferentes

* **Total Call centre complaint calls:** número de llamadas de queja al call center

* **Network type subscription in Month 1:** suscripción de tipo de red en el mes 1. Esto indica la suscripción de red preferida de un cliente, lo que puede indicar su tipo de dispositivo: servicio 2G o 3G

* **Network type subscription in Month 2:** igual que el anterior pero en el mes posterior

* **Churn Status**: el valor es 1 si el cliente abandona la compañía telefónica, 0 si permanece en ella

* **Most Loved Competitor network in Month 1:** qué otro proveedor de la competencia prefiere el cliente en el mes 1. En realidad es un conjunto de columnas, cada una enfocada en un proveedor particular

* **Most Loved Competitor network in Month 2:** qué otro proveedor de la competencia prefiere el cliente en el mes 2. En realidad es un conjunto de columnas, cada una enfocada en un proveedor particular

La variable a predecir es **Churn Status**: el valor es 1 si el cliente **abandona** la compañía, 0 si no la abandona.

La compañía también nos proporciona otra base de datos, **fuga_clientes_empresa_telefonica_explotacion.csv**, con información sobre clientes de los que no se sabe ahora mismo si van a permanecer o no en la compañía. Por tanto en esta segunda base de datos todos los valores de la columna **Churn Status** son missing values (NaN).

La compañía nos pide que proporcionemos los IDs de los 100 clientes de la base de datos de explotación que con mayor probabilidad vayan a abandonar la compañía. Para ello proporcionaremos como entregable a la compañía un archivo csv con una sola columna, **Customer ID**, y 100 filas que contengan los IDs de los clientes seleccionados.

El fichero **ejemplo_fichero_predicciones.csv** contiene un ejemplo con el formato solicitado para este archivo.

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs, make_moons, make_circles
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

In [11]:
datos_construccion = pd.read_csv("./fuga_clientes_empresa_telefonica_construccion.csv")
datos_explotacion  = pd.read_csv("./fuga_clientes_empresa_telefonica_explotacion.csv")

In [12]:
datos_construccion.columns

Index(['Customer ID', 'network_age', 'Customer tenure in month',
       'Total Spend in Months 1 and 2', 'Total SMS Spend', 'Total Data Spend',
       'Total Data Consumption', 'Total Unique Calls', 'Total Onnet spend',
       'Total Offnet spend', 'Total Call centre complaint calls',
       'Churn Status', 'Most Loved Competitor network in Month 1_0',
       'Most Loved Competitor network in Month 1_Mango',
       'Most Loved Competitor network in Month 1_PQza',
       'Most Loved Competitor network in Month 1_ToCall',
       'Most Loved Competitor network in Month 1_Uxaa',
       'Most Loved Competitor network in Month 1_Weematel',
       'Most Loved Competitor network in Month 1_Zintel',
       'Most Loved Competitor network in Month 2_Mango',
       'Most Loved Competitor network in Month 2_PQza',
       'Most Loved Competitor network in Month 2_ToCall',
       'Most Loved Competitor network in Month 2_Uxaa',
       'Most Loved Competitor network in Month 2_Weematel',
       'Most 

In [13]:
datos_explotacion.columns

Index(['Customer ID', 'network_age', 'Customer tenure in month',
       'Total Spend in Months 1 and 2', 'Total SMS Spend', 'Total Data Spend',
       'Total Data Consumption', 'Total Unique Calls', 'Total Onnet spend',
       'Total Offnet spend', 'Total Call centre complaint calls',
       'Churn Status', 'Most Loved Competitor network in Month 1_0',
       'Most Loved Competitor network in Month 1_Mango',
       'Most Loved Competitor network in Month 1_PQza',
       'Most Loved Competitor network in Month 1_ToCall',
       'Most Loved Competitor network in Month 1_Uxaa',
       'Most Loved Competitor network in Month 1_Weematel',
       'Most Loved Competitor network in Month 1_Zintel',
       'Most Loved Competitor network in Month 2_Mango',
       'Most Loved Competitor network in Month 2_PQza',
       'Most Loved Competitor network in Month 2_ToCall',
       'Most Loved Competitor network in Month 2_Uxaa',
       'Most Loved Competitor network in Month 2_Weematel',
       'Most 

In [14]:
# chequeo de que las columnas son exactamente las mismas en los dos ficheros:

sum(datos_construccion.columns != datos_explotacion.columns)

0

In [15]:
# la columna a predecir es "Churn Status"
# si es 1, el cliente se va de la compañía
# si es 0, el cliente se queda

# Esta columna se sabe en el dataset de construcción (ejemplos de clientes pasados):
datos_construccion["Churn Status"].values[:10]

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1])

In [16]:
# Sin embargo no se sabe en el dataset de explotación (clientes actuales):
datos_explotacion["Churn Status"].values[:10]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [17]:
# We delete 'Customer ID' and "Churn Status" because it does not allow strings ntor nulls

nombres_atrs = list(datos_construccion.columns)
nombres_atrs.remove('Churn Status')
nombres_atrs.remove('Customer ID')
print(nombres_atrs)
X = datos_construccion[nombres_atrs].values
y = datos_construccion['Churn Status'].values

['network_age', 'Customer tenure in month', 'Total Spend in Months 1 and 2', 'Total SMS Spend', 'Total Data Spend', 'Total Data Consumption', 'Total Unique Calls', 'Total Onnet spend', 'Total Offnet spend', 'Total Call centre complaint calls', 'Most Loved Competitor network in Month 1_0', 'Most Loved Competitor network in Month 1_Mango', 'Most Loved Competitor network in Month 1_PQza', 'Most Loved Competitor network in Month 1_ToCall', 'Most Loved Competitor network in Month 1_Uxaa', 'Most Loved Competitor network in Month 1_Weematel', 'Most Loved Competitor network in Month 1_Zintel', 'Most Loved Competitor network in Month 2_Mango', 'Most Loved Competitor network in Month 2_PQza', 'Most Loved Competitor network in Month 2_ToCall', 'Most Loved Competitor network in Month 2_Uxaa', 'Most Loved Competitor network in Month 2_Weematel', 'Most Loved Competitor network in Month 2_Zintel']


In [18]:
# We evaluate different methods that we can use to evaluate our database with the values obtained earlier 
# in the exercise "02-p4_IA.ipynb", that is max_depth = 5, n_neighbors = 35, hidden_layer_sizes = (150)
# We will obtain the global Score of each model as we did in said exercise

import warnings
warnings.filterwarnings('ignore')

scoreGlobal_DT = DecisionTreeClassifier(max_depth=5)
scores = cross_val_score(scoreGlobal_DT, X, y, cv=5)
print("Score global de DecisionTree: {:.2f} +/- {:.2f}".format(scores.mean(), scores.std()))

scoreGlobal_KNN = KNeighborsClassifier(n_neighbors=35)
scores = cross_val_score(scoreGlobal_KNN, X, y, cv=5)
print("Score global de k-NN: {:.2f} +/- {:.2f}".format(scores.mean(), scores.std()))

scoreGlobal_NB = GaussianNB()
scores = cross_val_score(scoreGlobal_NB, X, y, cv=5)
print("Score global de Naive Bayes Gaussiano: {:.2f} +/- {:.2f}".format(scores.mean(), scores.std()))

scoreGlobal_LR = LogisticRegression(solver='lbfgs')
scores = cross_val_score(scoreGlobal_LR, X, y, cv=5)
print("Score global de Regresion Logística: {:.2f} +/- {:.2f}".format(scores.mean(), scores.std()))

scoreGlobal_MLP = MLPClassifier(hidden_layer_sizes=(150), max_iter=1000, alpha=0)
scores = cross_val_score(scoreGlobal_MLP, X, y, cv=5)
print("Score global de redes neuronales: {:.2f} +/- {:.2f}".format(scores.mean(), scores.std()))

Score global de DecisionTree: 0.77 +/- 0.06
Score global de k-NN: 0.63 +/- 0.05
Score global de Naive Bayes Gaussiano: 0.62 +/- 0.02
Score global de Regresion Logística: 0.63 +/- 0.01
Score global de redes neuronales: 0.55 +/- 0.06


### Knowing that the DecisionTree is the best option (DecisionTree global score ≈ 0.77 +/- 0.06) to carry out the tests, we must extract the data that suits us to carry out the lost task.

In [22]:
# First we calculate the probability of hits that we have using the construccion_data file.

scoreGlobal_DT.fit(X, y)
predictions = scoreGlobal_DT.predict(X)

fail = 0
succ = 0
for i,churn in enumerate(predictions):
    if churn == y[i]:
        succ += 1
    else:
        fail += 1
        
print("Se van a ir", succ,"clientes de",succ+fail,"predicciones que hemos hecho.")
print("Hemos acertado con un {:.2f}% de probabilidad.".format((succ / (succ+fail))*100))

Se van a ir 971 clientes de 1110 predicciones que hemos hecho.
Hemos acertado con un 87.48% de probabilidad.


In [23]:
# Finally, we predict the number of coustumers that are soing to leave the company.

predictions = scoreGlobal_DT.predict(datos_explotacion[nombres_atrs].values)

stay = go = 0
for i,n in enumerate(predictions):
    if n == 1:
        go += 1
    else:
        stay += 1
        
print("Se van a ir", go,"clientes de",go+stay)
print("Sabemos que va a ser cierto con un {:.2f}% de probabilidad.".format((succ / (succ+fail))*100))

Se van a ir 114 clientes de 289
Sabemos que va a ser cierto con un 87.48% de probabilidad.


In [25]:
# Now we concatenate the ID of the client together with the prediction that we have obtained previously.
# With the help of pandas we put it in a matrix, of which we are only interested that the prediction we have
# obtained is equal to 1, that is, they leave the company.

results = {'Customer ID': datos_explotacion["Customer ID"].values,'Prediction': predictions}
results_df = pd.DataFrame(results, columns=['Customer ID','Prediction'])

results_df = results_df[results_df['Prediction']==1][['Customer ID']]
results_df.head()

,Customer ID
30,ADF1560
31,ADF1561
33,ADF1563
34,ADF1564
35,ADF1567


In [26]:
# we copy the matrix in the .csv

results_df.head(100).to_csv('ClientesQueSeVan.csv', index=False)